In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import pickle
import random

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zroy1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [43]:
n = 1600000 #number of records in file
s = 100000 #desired sample size
filename = "twitter_tweets.csv"
skip = sorted(random.sample(range(n),n-s))
positivity = pd.read_csv(filename, skiprows=skip, encoding='latin-1')
positivity

,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
0,0,1467812771,Mon Apr 06 22:20:19 PDT 2009,NO_QUERY,robrobbierobert,"@octolinz16 It it counts, idk why I did either..."
1,0,1467813137,Mon Apr 06 22:20:25 PDT 2009,NO_QUERY,armotley,about to file taxes
2,0,1467815923,Mon Apr 06 22:21:07 PDT 2009,NO_QUERY,fatkat309,some1 hacked my account on aim now i have to ...
3,0,1467817502,Mon Apr 06 22:21:32 PDT 2009,NO_QUERY,Tmttq86,@fleurylis I don't either. Its depressing. I d...
4,0,1467819650,Mon Apr 06 22:22:05 PDT 2009,NO_QUERY,antzpantz,@Viennah Yay! I'm happy for you with your job!...
...,...,...,...,...,...,...
199994,4,2193577616,Tue Jun 16 08:38:51 PDT 2009,NO_QUERY,twisted4ddub,@_CrC_ mornin.. I'm enjoying a beautiful morni...
199995,4,2193578982,Tue Jun 16 08:38:58 PDT 2009,NO_QUERY,LISKFEST,"if ur the lead singer in a band, beware fallin..."
199996,4,2193579087,Tue Jun 16 08:38:58 PDT 2009,NO_QUERY,marhgil,@tarayqueen too much ads on my blog.
199997,4,2193579284,Tue Jun 16 08:38:59 PDT 2009,NO_QUERY,AgustinaP,@rmedina @LaTati Mmmm That sounds absolutely ...


In [44]:
labels = ["sentiment", "id", "date", "flag", "user", "text"]
positivity.columns = labels
positivity

,sentiment,id,date,flag,user,text
0,0,1467812771,Mon Apr 06 22:20:19 PDT 2009,NO_QUERY,robrobbierobert,"@octolinz16 It it counts, idk why I did either..."
1,0,1467813137,Mon Apr 06 22:20:25 PDT 2009,NO_QUERY,armotley,about to file taxes
2,0,1467815923,Mon Apr 06 22:21:07 PDT 2009,NO_QUERY,fatkat309,some1 hacked my account on aim now i have to ...
3,0,1467817502,Mon Apr 06 22:21:32 PDT 2009,NO_QUERY,Tmttq86,@fleurylis I don't either. Its depressing. I d...
4,0,1467819650,Mon Apr 06 22:22:05 PDT 2009,NO_QUERY,antzpantz,@Viennah Yay! I'm happy for you with your job!...
...,...,...,...,...,...,...
199994,4,2193577616,Tue Jun 16 08:38:51 PDT 2009,NO_QUERY,twisted4ddub,@_CrC_ mornin.. I'm enjoying a beautiful morni...
199995,4,2193578982,Tue Jun 16 08:38:58 PDT 2009,NO_QUERY,LISKFEST,"if ur the lead singer in a band, beware fallin..."
199996,4,2193579087,Tue Jun 16 08:38:58 PDT 2009,NO_QUERY,marhgil,@tarayqueen too much ads on my blog.
199997,4,2193579284,Tue Jun 16 08:38:59 PDT 2009,NO_QUERY,AgustinaP,@rmedina @LaTati Mmmm That sounds absolutely ...


In [45]:
# 0: negative
# 2: neutral
# 4: positive
positivity["sentiment"].value_counts()

0    100129
4     99870
Name: sentiment, dtype: int64

In [46]:
positivity["sentiment"] = positivity["sentiment"].replace(4, 1)
positivity["sentiment"].value_counts()

0    100129
1     99870
Name: sentiment, dtype: int64

In [47]:
X = positivity["text"]
y = positivity["sentiment"]

In [48]:
messages = []
for text in X:
    text = re.sub(r'[!@#$(),\n"%^*?\:;~`0-9]', ' ', str(text))
    text = text.lower()
    messages.append(text)
messages

[' octolinz   it it counts  idk why i did either. you never talk to me anymore ',
 'about to file taxes ',
 'some  hacked my account on aim  now i have to make a new one',
 " fleurylis i don't either. its depressing. i don't think i even want to know about the kids in suitcases. ",
 " viennah yay  i'm happy for you with your job  but that also means less time for me and you... ",
 " localtweeps wow  tons of replies from you  may have to unfollow so i can see my friends' tweets  you're scrolling the feed a lot. ",
 'i need a hug ',
 ' oanhlove i hate when that happens... ',
 " starrbby too bad i won't be around i lost my job and can't even pay my phone bill lmao aw shucks ",
 'poor cameron  the hills  ',
 " stark you don't follow me  either  and i work for you ",
 ' body of missing northern calif. girl found  police have found the remains of a missing northern california girl .. http //tr.im/imji',
 ' marykatherine_q i know  i heard it this afternoon and wondered the same thing. moscow 

In [49]:
stop_words = set(stopwords.words('english'))
for i in range(len(messages)):
    if i % 100000 == 0:
        print(i)
    word_tokens = word_tokenize(messages[i], language='english', preserve_line=True)
    message = " ".join(word for word in word_tokens if not word.lower() in stop_words)
    messages[i] = message

0
100000


In [41]:
np.array(messages).shape

(1599999,)

In [50]:
# vectorize
cv = CountVectorizer(token_pattern=r'[^\s]+')
X = cv.fit_transform(messages).toarray()
X.shape

MemoryError: Unable to allocate 257. GiB for an array with shape (199999, 172227) and data type int64

In [22]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [23]:
model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

In [24]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7385

In [ ]:
def prediction(text, model):
    text = text.lower()
    x = cv.transform([text]).toarray()
    lang = model.predict(x)
    lang = labelEncoder.inverse_transform(lang)
    print(lang[0])

In [ ]:
test = "Roy im going to murder you in your house"
prediction(test, model)